In [1]:
import pandas as pd
import numpy as np

In [2]:
read_from_path = 'D:\\CS\\Projects\\p4k\\reviewListUnClean'
df = pd.read_csv(read_from_path)

In [3]:
df.head()

,title,artist,year,score,best new music,best new reissue,genre,label,author,abstract,review,word count,char count,publication date,url
0,Cracker Island,Gorillaz,2023,6.5,0,0,Electronic / Rock,NaN,Ben Cardew,"Stevie Nicks, Thundercat, Tame Impala, and Bad...",As a band supposedly made up of cartoon charac...,741,4752,2023-02-23T05:02:00,https://pitchfork.com//reviews/albums/gorillaz...
1,Mourning Due,Nappy Nina,2023,6.2,0,0,Rap,LucidHaus,Stephen Kearse,The hushed flows from the Brooklyn-based rappe...,"If Nappy Nina were a comic book character, her...",631,3966,2023-02-23T05:01:00,https://pitchfork.com//reviews/albums/nappy-ni...
2,Desire Pathway,Screaming Females,2023,6.8,0,0,Rock,Don Giovanni,Ian Cohen,The New Jersey band turns in another solid alb...,Screaming Females don't immediately seem like ...,716,4492,2023-02-23T05:00:00,https://pitchfork.com//reviews/albums/screamin...
3,Quest for Fire,Skrillex,2023,7.7,0,0,Electronic,Owsla,Chal Ravens,The dubstep disruptor returns with two albums—...,Skrillex is what tech folks would call a disru...,1540,9404,2023-02-22T05:03:00,https://pitchfork.com//reviews/albums/skrillex...
4,Don’t Get Too Close,Skrillex,2023,7.3,0,0,Electronic,Owsla,Chal Ravens,The dubstep disruptor returns with two albums—...,Skrillex is what tech folks would call a disru...,1540,9404,2023-02-22T05:03:00,https://pitchfork.com//reviews/albums/skrillex...


In [4]:
df.shape

(26436, 15)

In [5]:
###########################################################################################################################

In [6]:
# DTypes

In [7]:
###########################################################################################################################

In [8]:
df.dtypes

title                object
artist               object
year                 object
score               float64
best new music        int64
best new reissue      int64
genre                object
label                object
author               object
abstract             object
review               object
word count            int64
char count            int64
publication date     object
url                  object
dtype: object

In [9]:
df['year reissue'] = None
df['best new music'] = df['best new music'].astype(bool)
df['best new reissue'] = df['best new reissue'].astype(bool)


df.rename(columns={'best new music':'isBNM'}, inplace = True)
df.rename(columns={'best new reissue':'isBNR'}, inplace = True)
df.rename(columns={'publication date':'publication datetime'}, inplace = True)

In [13]:
###########################################################################################################################

In [14]:
# Isna

In [15]:
###########################################################################################################################

In [17]:
# Title - Issue with how for loop in scrapper works 
# Artist - Scrapper was too specific, should work next time
# Year - Some pages dont have year, will assume if year is NA then the year of publication is the year of release
# Genre - Some pages just dont have genre, will just have to ignore these for analysis
# Label - Same as Genre
# Abstract - Some abstracts had different html characteristics, should be fixed for next scrape
# Review - Joke reviews
df.isna().sum()

title                       1
artist                      1
year                      414
score                       0
isBNM                       0
isBNR                       0
genre                    2409
label                    3512
author                      0
abstract                  575
review                      2
word count                  0
char count                  0
publication datetime        0
url                         0
year reissue            26436
dtype: int64

In [18]:
tempDF = df[df['url'].duplicated()]
df = df.drop(tempDF[tempDF['title'].isna()].index)

In [19]:
columns = df.drop(['year', 'year reissue'], axis=1).columns

for column in columns:
    df[column] = df[column].fillna("")

In [20]:
###########################################################################################################################

In [21]:
# New Columns

In [22]:
###########################################################################################################################

In [23]:
# Splitting the Date into a published date column and published time
df[['publication date', 'publication time']] = df['publication datetime'].str.split('T', expand=True)

In [24]:
df['publication year'] = df['publication datetime'].str.split('-', expand=True)[0]

In [25]:
s = df[df['year'].isna()]['publication datetime'].str.split('-', 1).str[0].astype(int)
df.loc[s.index, 'year'] = s

C:\Users\dttan\AppData\Local\Temp\ipykernel_21352\2015507722.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  s = df[df['year'].isna()]['publication datetime'].str.split('-', 1).str[0].astype(int)


In [26]:
# Creating a isReissue column
df['isReissue'] = False

s = (df[df['year'].str.contains('/', na=False)]['publication datetime'].str.split('-', 1).str[0].astype(int) > df[df['year'].str.contains('/', na=False)]['year'].str.split('/').str[0].astype(int))
df.loc[s.index[s],'isReissue'] = True

C:\Users\dttan\AppData\Local\Temp\ipykernel_21352\2449848172.py:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  s = (df[df['year'].str.contains('/', na=False)]['publication datetime'].str.split('-', 1).str[0].astype(int) > df[df['year'].str.contains('/', na=False)]['year'].str.split('/').str[0].astype(int))


In [27]:
df['isLookback'] = False
df.loc[df[df['year'] != df['publication year']].index, 'isLookback'] = True

In [28]:
s = df[df['year'].str.contains('/', na=False)]['year'].str.split('/').str[0].astype(int)
s1 = df[df['year'].str.contains('/', na=False)]['year'].str.split('/').str[1].astype(int)

df.loc[s.index, 'year'] = s
df.loc[s1.index, 'year reissue'] = s1

In [29]:
# Creating a isEP column
df['isEP'] = False

s = df['title'].str.contains('\WEP')
df.loc[s.index[s],'isEP'] = True

In [30]:
#s = df[df['genre'].str.contains(' / ', na=False)]['genre'].str.split(' / ')
s = df['genre'].str.split(' / ')
df.loc[s.index,'genre'] = s

In [31]:
# Looking for potential words to indicate not a new release in the title of the album

s=df['title'].str.contains(' Legacy| Deluxe| Edition| Compliation| reissued| Reissued| Remixes| Remix| B-sides| Singles|xCD| Box| \d*-\d*| Greatest Riffs')
df.loc[s.index[s], 'isReissue'] = True
df.loc[s.index[s], 'isBNM'] = False

False    25722
True       714
Name: title, dtype: int64

In [32]:
# Looking for potential words to indicate not a new release in the abstract of the review

s = df['abstract'].str.contains(r'([Rr]eissue)|([Rr]eissued)|([Rr]eissuing)|([Bb]oxset)|([Bb]ox set)|([Rr]emastered)|xCD|singles collections|([Bb]est-of)')
df.loc[s.index[s], 'isReissue'] = True
df.loc[s.index[s], 'isBNM'] = False  

C:\Users\dttan\AppData\Local\Temp\ipykernel_21352\2868151497.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  s = df['abstract'].str.contains(r'([Rr]eissue)|([Rr]eissued)|([Rr]eissuing)|([Bb]oxset)|([Bb]ox set)|([Rr]emastered)|xCD|singles collections|([Bb]est-of)')


False    24791
True      1645
Name: abstract, dtype: int64

In [35]:
tempDf = df[df['isReissue']]
tempDf = tempDf[(tempDf["year"] == tempDf["publication year"])]
tempDf = tempDf[tempDf['abstract'].str.contains(r' \d\d\d\d')]

s = tempDf['abstract'].str.extract(r'( \d\d\d\d)')
df.loc[s.index,'year'] = s

In [37]:
###########################################################################################################################

In [38]:
#Columns Reorder

In [39]:
###########################################################################################################################

In [41]:
df = df[['title', 'artist', 'year', 'score', 'genre', 'label', 'isBNM', 'isBNR', 'isEP', 'isReissue', 'isLookback',
         'year reissue', 'author', 'abstract', 'review', 'word count', 'char count',
         'publication datetime', 'publication year', 'publication date', 'publication time', 'url']]

In [44]:
# Path to save to
save_to_path = 'D:\\CS\\Projects\\p4k\\reviewList.csv'

df.to_csv(save_to_path, index=False)